In [1]:
from utils import (
    get_cross_validation_results,
    get_final_model_performance,
    TransactionDataset,
    TuneHyperParams,
)

from constants import Columns, ModelConstants, ResamplingStrategy

In [2]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    FunctionTransformer,
    Normalizer,
    PowerTransformer,
    RobustScaler
)
from sklearn.compose import make_column_transformer, make_column_selector

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, ADASYN


In [3]:
data = TransactionDataset().get_training_test_split()

In [4]:
log_transform = FunctionTransformer(
    func=np.log1p, inverse_func=np.expm1, check_inverse=False
)

order_mapping = [['0', '0.2', '0.4', '0.6', '0.8', '1']]

norm_and_one_hot_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (Normalizer(), make_column_selector(dtype_include="number"))
)

log_and_ordinal_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (log_transform, make_column_selector(dtype_include="number"))
)

power_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (PowerTransformer(), make_column_selector(dtype_include="number"))
)

robust_transformation = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY]),
    (RobustScaler(), make_column_selector(dtype_include="number"))
)

In [5]:
knn_classifier = KNeighborsClassifier(
    n_jobs=ModelConstants.N_JOBS
)

In [6]:
smote_sampling = SMOTE(random_state=ModelConstants.RANDOM_STATE)
adasyn_sampling = ADASYN(random_state=ModelConstants.RANDOM_STATE)

In [7]:
column_transforms_only_model = Pipeline(
    [
        ("column_transformation", log_and_ordinal_transforms),
        ("sampling", smote_sampling),
        ("knn", knn_classifier)
    ]
)

In [8]:
tuning_params = {
    "column_transformation": [
        log_and_ordinal_transforms,
        norm_and_one_hot_transforms,
        power_transforms,
        robust_transformation,
        "passthrough"
    ],

    "sampling": [
        smote_sampling,
        adasyn_sampling,
    ],
    "sampling__sampling_strategy": [
        ResamplingStrategy.MINORITY_ONLY,
        ResamplingStrategy.ALL,
        0.2,
        0.3,
        0.4,
        0.5,
        0.6,
        0.7
    ],

    "knn__n_neighbors": [5, 7, 9, 11, 13, 15, 17, 19, 21, 25],
    "knn__weights": ["uniform", "distance"],
}

In [ ]:
best_model = (
    TuneHyperParams()
    .random_grid_search(
     column_transforms_only_model,
     tuning_params
    )
    .fit_model(
        data.TRAINING.predictors,
        data.TRAINING.outcome
    )
    .get_best_model()
)

In [ ]:
get_cross_validation_results(best_model, data.TRAINING.predictors, data.TRAINING.outcome)

In [ ]:
get_final_model_performance(best_model, data.TRAINING, data.TESTING)